<a href="https://colab.research.google.com/github/MahdiRahbar/NLP_SLU/blob/main/sentiment/WelshSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
import random
import math

In [127]:
import os
import re
from collections import defaultdict
fn = os.path.join("..","data","train-v2.tsv")
handle = open(fn, "r")
res = list()
tweets = list()
for i in handle:
    entry = i.strip().split("\t")
    res.append(int(entry[0]))
    tweets.append(entry[1])

In [130]:
# i tried
def reformat_tweets(tweets):
    #get rid of mentions, links, hashtags and numbers
    tweets2 = map(lambda x: re.sub("(@USER|\{URL\}|#\w*|[0-9]+)", "", x).strip(), tweets)
    #lowercase everything
    tweets2 = map(lambda x: x.lower(), tweets2)
    #tokenize
    tweets2 = [t.split(" ") for t in tweets2]
    #now remove all the punctuations that is not part of emoticon (meaninng PUNC following some letter)
    tweets_reformatted = []
    for tweet in tweets2:
        clean_tweet = []
        for word in tweet:
            if bool(re.search("\w[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]", word)):
                word = re.sub("\W","", word )
            clean_tweet.append(word)
        tweets_reformatted.append(clean_tweet)
    return tweets_reformatted
def make_bag_of_words(tweets):
    bag_of_words = defaultdict(int)  # key-word, val-count
    for tweet in tweets:
        for word in tweet:
            bag_of_words[word] += 1
    return bag_of_words


In [153]:
from collections import defaultdict
class NaiveBayes():
    def __init__(self, xVec:list, yVec:list, vocabs:defaultdict(int)):
        self.xVec = xVec
        self.yVec = yVec
        self.classes = set(yVec)

        self.vocabs = vocabs

        self.count_by_class = defaultdict(int)
        self.bigdoc_by_class = defaultdict(lambda: defaultdict(int))
       
        self.prior_prob = defaultdict(float)
        self.likelihood = defaultdict(lambda: defaultdict(float))

    def sep_by_class(self):
        # assuming no missing data len(xVec) == len(yVec).
        for i in range(len(self.yVec)):
            c = self.yVec[i] #class of the current tweet
            tweet = self.xVec[i]
            self.count_by_class[c] += 1
            for word in tweet:
                self.bigdoc_by_class[c][word] +=1
    
    def train_bayes(self, smoothing = None):
        for c in self.classes:

            self.prior_prob[c] = math.log( (self.count_by_class[c]+1)/(len(self.yVec) +1) ) #calculate prior
            all_words = sum(self.bigdoc_by_class[c][w] for w in self.bigdoc_by_class[c])
            for word in self.vocabs: 
                # calculate likelihood
                numerator = self.bigdoc_by_class[c][word] + 1
                denominator = all_words + len(self.vocabs)
                self.likelihood[word][c] = math.log(numerator/denominator)
    def test_bayes(self, testX):
        bestClass = None
        for c in self.classes:
            prob = self.prior_prob[c]
            for w in testX:
                if w in self.vocabs:
                    prob += self.likelihood[w][c]
            if bestClass == None or prob > bestClass:
                bestClass = c

        return bestClass


In [154]:
#divide train-test
import random
random.seed(42)
train_indexes = random.sample( list(range(80000)), 64000)
tweets_reformatted = reformat_tweets(tweets)

trainX = [tweets_reformatted[i] for i in train_indexes]
trainY = [res[i] for i in train_indexes]
vocabs = make_bag_of_words(trainX)
nb = NaiveBayes(trainX, trainY, vocabs)
nb.sep_by_class()
nb.train_bayes()
nb.test_bayes(trainX[0])


0

In [176]:
nb.prior_prob

defaultdict(float, {0: -0.694601339632223, 1: -0.6916639293141402})

In [167]:
trainY[1]

0

In [ ]:
# this should return either 0 (negative sentiment) or 1 (positive sentiment)

def predict_from_scratch(tweet):
  return

In [ ]:
# this should return either 0 (negative sentiment) or 1 (positive sentiment)
def predict_anything_goes(tweet):
  # do something complicated here
  return random.randint(0,1)

In [ ]:
def evaluate():
  total = 0
  correct_from_scratch = 0
  correct_anything_goes = 0
  testfile = open('test.tsv', 'r')
  for line in testfile:
    total += 1
    pieces = line.rstrip("\n").split("\t")
    if predict_from_scratch(pieces[1]) == int(pieces[0]):
      correct_from_scratch += 1
    if predict_anything_goes(pieces[1]) == int(pieces[0]):
      correct_anything_goes += 1
  return (correct_from_scratch/total, correct_anything_goes/total)
  

In [ ]:
evaluate()

(0.5096, 0.4987)